In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
import tensorflow as tf

In [2]:
from google.colab import files

# Upload the file
uploaded = files.upload()


Saving Sarcasm_Headlines_Dataset_v2.json to Sarcasm_Headlines_Dataset_v2 (1).json


In [3]:
import pandas as pd

# Replace 'data.json' with your file name if different
df = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines=True)


In [4]:
df

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...


In [5]:
del df['article_link']

In [6]:
import nltk
nltk.download('stopwords')


stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['headline']=df['headline'].apply(denoise_text)

<ipython-input-7-ba69b7b9ea42>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [8]:
words = []
for i in df.headline.values:
    words.append(i.split())
words[:5]

[['thirtysomething',
  'scientists',
  'unveil',
  'doomsday',
  'clock',
  'hair',
  'loss'],
 ['dem',
  'rep.',
  'totally',
  'nails',
  'congress',
  'falling',
  'short',
  'gender,',
  'racial',
  'equality'],
 ['eat', 'veggies:', '9', 'deliciously', 'different', 'recipes'],
 ['inclement', 'weather', 'prevents', 'liar', 'getting', 'work'],
 ['mother',
  'comes',
  'pretty',
  'close',
  'using',
  'word',
  "'streaming'",
  'correctly']]

In [9]:
import gensim
#Dimension of vectors we are generating
EMBEDDING_DIM = 200

#Creating Word Vectors by Word2Vec Method (takes time...)
w2v_model = gensim.models.Word2Vec(sentences=words, vector_size=EMBEDDING_DIM, window=5, min_count=1)

In [10]:
tokenizer = text.Tokenizer(num_words=35000)
tokenizer.fit_on_texts(words)
tokenized_train = tokenizer.texts_to_sequences(words)
x = sequence.pad_sequences(tokenized_train, maxlen = 20)

In [11]:
vocab_size = len(tokenizer.word_index) + 1

In [12]:
# Function to create weight matrix from word2vec gensim model
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix

In [13]:
x_train,x_test,y_train,y_test = train_test_split(df.headline,df.is_sarcastic, test_size = 0.3 , random_state = 0)

In [14]:
max_features = 35000
maxlen = 200

In [15]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)


In [16]:
tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [17]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [18]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
# Tokenize and preprocess text
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and token.isalpha()]
    return tokens

# Apply preprocessing
df['clean_headline'] = df['headline'].apply(preprocess_text)

# Train Word2Vec model
model = Word2Vec(sentences=df['clean_headline'], vector_size=100, window=5, min_count=1, workers=4)

# Transform headlines into vectors
def headline_to_vector(headline):
    vector = []
    for word in headline:
        if word in model.wv:
            vector.append(model.wv[word])
    return sum(vector) / len(vector) if vector else [0]*100  # Return a zero vector if no words found

df['headline_vector'] = df['clean_headline'].apply(headline_to_vector)

# Split data for training
X_train, X_test, y_train, y_test = train_test_split(df['headline_vector'].tolist(), df['is_sarcastic'], test_size=0.2)

# Train classifier (using Logistic Regression as an example)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict and evaluate
predictions = classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.6114605171208944


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import gensim.downloader as api
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

# Load pre-trained GloVe word vectors
word_vectors = api.load("glove-wiki-gigaword-100")

# Function to get vector representation for a headline
def get_headline_vector(headline):
    vector = []
    for word in headline:
        word = word.lower()
        if word not in stop_words and word.isalpha() and word in word_vectors:
            vector.append(word_vectors[word])
    return sum(vector) / len(vector) if vector else [0]*100  # Return a zero vector if no valid words found

# Apply preprocessing to headlines
def preprocess_text(text):
    tokens = word_tokenize(text)
    return [token for token in tokens if token.lower() not in stop_words and token.isalpha()]

# Apply preprocessing to headlines
df['clean_headline'] = df['headline'].apply(preprocess_text)

# Get GloVe vectors for each headline
df['glove_vector'] = df['clean_headline'].apply(get_headline_vector)

# Split data for training
X_train, X_test, y_train, y_test = train_test_split(df['glove_vector'].tolist(), df['is_sarcastic'], test_size=0.2)

# Train classifier (using Support Vector Machine as an example)
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

# Predict and evaluate
predictions = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.7010831586303284


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# Convert headlines to text
df['headline_text'] = df['clean_headline'].apply(lambda x: ' '.join(x))

# TF-IDF Vectorization
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['headline_text'])

# Split data for training
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df['is_sarcastic'], test_size=0.2)

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)  # You can adjust hyperparameters here
rf_classifier.fit(X_train, y_train)

# Predict and evaluate
predictions = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.7688679245283019
